# Main Demo
## Introduction

This Jupyter Notebook contains the code to reproduce the results presented in our paper "A Mechanistic Interpretation of Syllogistic Reasoning in Auto-Regressive Language Models".

### Contents:
1. Setup
2. Dataset
3. Intervention (Figure 4)
4. Evaluation (Figure 5)
5. Transferability (Figure 6, Table 2, Figure 3)

Note: This demo is provided to ensure transparency and reproducibility of our research. If you encounter any issues or have questions, please contact the authors.

# 1. Setup

+ python >= 3.9.18
+ pytorch >= 2.2.0
+ plotly >= 5.19.0

In [ ]:
# Installation of transformer_lens
!pip install transformer_lens
!pip install circuitsvis

In [1]:
# Import libraries
import torch as t
from torch import Tensor
import random
from functools import partial
import importlib
import circuitsvis as cv
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

from prepare_dataset import SyllogismDataset
import helper_functions as h
t.set_grad_enabled(False)

/Users/geonhee/miniconda3/envs/textmining/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Device Setup
device = t.device("cuda" if t.cuda.is_available() else "cpu")
device = t.device("mps") # if machine has mps setup

In [3]:
# Import the model-medium Through HookedTransformer
model_size = ["gpt2-small", "gpt2-medium", "gpt2-large", "gpt2-xl"]
current_model = model_size[1] # medium

model = HookedTransformer.from_pretrained(
    current_model,
    fold_ln=False,
    device = device
)

Loaded pretrained model gpt2-medium into HookedTransformer


In [4]:
# label setup for sequence
labels = [
    "BEGIN",
    "s_1",
    "s_1 -> m_1",
    "m_1",
    "m_1 -> m_2",
    "m_2",
    "m_2 -> p",
    "p",
    "p -> s_2",
    "s_2",
    "END"
]

# 2. Dataset

In [5]:
from prepare_dataset import SyllogismDataset
import helper_functions as h

In [6]:
# Ensure the Dataset folder and dataset_generator.py file is in the same directory as this notebook
# This file contains the custom data generation functions used in our experiments
# 90 samples for each dataset, seed is set to 317 for reproducibility
N = 90
seed = 317

In [7]:
# Symbolic

s_dataset = SyllogismDataset(
            N=N/6, # because of permutation
            seed=seed,
            device=device,
            type='symbolic',
            template_type='AAA1'
        )
for s, l in zip(s_dataset.sentences[:6], s_dataset.labels[:6]):
    print(f'{s} => {l}')

All O are A. All A are W. Therefore, all O are =>  W
All O are W. All W are A. Therefore, all O are =>  A
All A are O. All O are W. Therefore, all A are =>  W
All A are W. All W are O. Therefore, all A are =>  O
All W are O. All O are A. Therefore, all W are =>  A
All W are A. All A are O. Therefore, all W are =>  O


In [8]:
len(s_dataset.sentences)

90

In [9]:
# Belief-Consistent
bc_dataset = SyllogismDataset(
            N=N,
            seed=seed,
            device=device,
            type='consistent',
            template_type='AAA1'
        )
for s, l in zip(bc_dataset.sentences[:6], bc_dataset.labels[:6]):
    print(f'{s} => {l}')

All men are humans. All humans are mortal. Therefore, all men are =>  mortal
All dens are units. All units are parts. Therefore, all dens are =>  parts
All pets are animals. All animals are organisms. Therefore, all pets are =>  organisms
All openings are starts. All starts are beginnings. Therefore, all openings are =>  beginnings
All fronts are sides. All sides are surfaces. Therefore, all fronts are =>  surfaces
All fences are boundaries. All boundaries are lines. Therefore, all fences are =>  lines


In [10]:
# Belief-Inconsistent
bi_dataset = SyllogismDataset(
            N=N,
            seed=seed,
            device=device,
            type='inconsistent',
            template_type='AAA1'
        )
for s, l in zip(bi_dataset.sentences[:6], bi_dataset.labels[:6]):
    print(f'{s} => {l}')

All patients are cases. All cases are arguments. Therefore, all patients are =>  arguments
All allegations are claims. All claims are rights. Therefore, all allegations are =>  rights
All wards are people. All people are judges. Therefore, all wards are =>  judges
All losers are people. All people are racists. Therefore, all losers are =>  racists
All aliens are people. All people are parents. Therefore, all aliens are =>  parents
All expectations are feelings. All feelings are sensitivity. Therefore, all expectations are =>  sensitivity


# 3. Empirical Evaluation Part

## Localization of Transitive Reasoning Mechanisms

In [11]:
# corruption function setup
ALPHABET_LIST = [' A', ' B', ' C', ' D', ' E', ' F', ' G', ' H', ' I', ' J', ' K', ' L', ' M', ' N', ' O', ' P', ' Q', ' R', ' S', ' T', ' U', ' V', ' W', ' X', ' Y', ' Z']
NUMBER_LIST = [' 1', ' 2', ' 3', ' 4', ' 5',' 6',' 7',' 8',' 9']
def middle_term_corrupt(prompts, As, Bs, labels, prompt_type):
    if prompt_type == 'symbolic':
        candidates = ALPHABET_LIST
    elif prompt_type == 'numeric':
        candidates = NUMBER_LIST
    elif prompt_type == 'non-symbolic':
        candidates = bc_dataset.A + bc_dataset.B

    corrupted_prompts = []

    for i in range(len(prompts)):
        prompt = prompts[i]
        label = labels[i]
        corrupted_labels = []
        A = As[i]
        B = Bs[i]

        new_list = list(filter(lambda x: x not in [A, B, label], candidates))
        target = random.choice(new_list)
        corrupted_prompt = prompt.replace('All' + B , 'All'+ target)
        corrupted_prompts.append(corrupted_prompt)
        corrupted_labels.append(target)

    return corrupted_prompts


def all_term_corrupt(prompts, As, Bs, labels, prompt_type):
    if prompt_type == 'symbolic':
        candidates = ALPHABET_LIST
    elif prompt_type == 'numeric':
        candidates = NUMBER_LIST
    elif prompt_type == 'non-symbolic':
        candidates = bc_dataset.A + bc_dataset.B

    corrupted_prompts = []
    for i in range(len(prompts)):
        prompt = prompts[i]
        label = labels[i]
        A = As[i]
        B = Bs[i]
        new_list = list(filter(lambda x: x not in [A, B, label], candidates))
        target = random.sample(new_list, 3)
        corrupted_prompt = 'All' + target[0] + ' are' + target[1] + '. All' + target[1] + ' are' + target[2] + '. Therefore, all' + target[0] + ' are'
        corrupted_prompts.append(corrupted_prompt)
    return corrupted_prompts



In [12]:
# answer pair setting (p, m)
s_prompts = s_dataset.sentences
s_labels = s_dataset.labels
s_second_labels = s_dataset.B
s_As = s_dataset.A
s_Bs = s_dataset.B

s_answers = list(zip(s_labels, s_second_labels))
s_answer_tokens = t.concat([
    model.to_tokens(names, prepend_bos=False).T for names in s_answers
])
for prompt, (p, m) in zip(s_prompts[:6], s_answers):
    print(prompt, p, m)

All O are A. All A are W. Therefore, all O are  W  A
All O are W. All W are A. Therefore, all O are  A  W
All A are O. All O are W. Therefore, all A are  W  O
All A are W. All W are O. Therefore, all A are  O  W
All W are O. All O are A. Therefore, all W are  A  O
All W are A. All A are O. Therefore, all W are  O  A


In [13]:
# Corruption and tokenization
s_corrupted_prompts = middle_term_corrupt(s_prompts, s_As, s_Bs, s_labels, 'symbolic')

s_tokens = model.to_tokens(s_prompts, prepend_bos=False).to(device)
s_corrupted_tokens = model.to_tokens(s_corrupted_prompts, prepend_bos=False).to(device)

In [14]:
# Compute logits for interventions
s_clean_logits, s_clean_cache = model.run_with_cache(s_tokens)
s_corrupted_logits, s_corrupted_cache = model.run_with_cache(s_corrupted_tokens)

s_clean_logit_diff = h.compute_logit_diff(s_clean_logits, s_answer_tokens)
print(f"Clean logit diff: {s_clean_logit_diff:.4f}")

s_corrupted_logit_diff = h.compute_logit_diff(s_corrupted_logits, s_answer_tokens)
print(f"Corrupted logit diff: {s_corrupted_logit_diff:.4f}")

Clean logit diff: 0.3691
Corrupted logit diff: -0.4470


### Figure 4(a) attention head output patching

In [26]:
s_attn_denoising = h.patching_attention(model, s_corrupted_tokens, s_clean_cache, h.metric_denoising, s_answer_tokens, s_clean_logit_diff, s_corrupted_logit_diff, 'z', device)

100%|██████████| 24/24 [00:53<00:00,  2.23s/it]


In [28]:
h.plot_attn(h.normalise_tensor(s_attn_denoising), labels)

### Figure 4(b) residual stream patching

In [29]:
s_resid_denoising = h.patching_residual(model, s_corrupted_tokens, s_clean_cache, h.metric_denoising, s_answer_tokens, s_clean_logit_diff, s_corrupted_logit_diff, device)

100%|██████████| 24/24 [00:50<00:00,  2.10s/it]


In [30]:
h.plot_residual(h.normalise_tensor(h.resize_all(s_resid_denoising)), labels)

### Figure 4(c) OV logit lens

In [32]:
token_alpha = model.to_tokens(ALPHABET_LIST, prepend_bos=False).to(device)
token_alpha = [ element[0].item() for element in token_alpha ]

In [33]:
deductive_head_ov = model.OV
ev = model.W_E
uev = model.W_U
ov_circuit = ev.cpu()[token_alpha, : ] @ deductive_head_ov.AB.cpu() @ uev.cpu()[:, token_alpha]

In [34]:
# OV circuit for head 11.10
h.plot_attn(h.normalise_tensor(ov_circuit[11, 10, : , :]), ALPHABET_LIST)

## Localization of Term-Related Information Flow

In [35]:
# Corruption and tokenization
s_corrupted_prompts = all_term_corrupt(s_prompts, s_As, s_Bs, s_labels, 'symbolic')

s_tokens = model.to_tokens(s_prompts, prepend_bos=False).to(device)
s_corrupted_tokens = model.to_tokens(s_corrupted_prompts, prepend_bos=False).to(device)

In [36]:
# Compute logits for interventions
s_clean_logits, s_clean_cache = model.run_with_cache(s_tokens)
s_corrupted_logits, s_corrupted_cache = model.run_with_cache(s_corrupted_tokens)

s_clean_logit_diff = h.compute_logit_diff(s_clean_logits, s_answer_tokens)
print(f"Clean logit diff: {s_clean_logit_diff:.4f}")

s_corrupted_logit_diff = h.compute_logit_diff(s_corrupted_logits, s_answer_tokens)
print(f"Corrupted logit diff: {s_corrupted_logit_diff:.4f}")

Clean logit diff: 0.3691
Corrupted logit diff: 0.0244


### Figure 4(d) residual stream patching

In [37]:
s_resid_denoising2 = h.patching_residual(model, s_corrupted_tokens, s_clean_cache, h.metric_denoising, s_answer_tokens, s_clean_logit_diff, s_corrupted_logit_diff, device)

100%|██████████| 24/24 [00:50<00:00,  2.10s/it]


In [38]:
h.plot_residual(h.normalise_tensor(h.resize_all(s_resid_denoising2)), labels)

# 4. Circuit Evaluation

### Figure 5(a) correctness of the circuit

In [39]:
# necessity of symbolic
necessity_score = h.necessity_check(model, s_labels, s_tokens, s_answer_tokens, s_clean_logit_diff, 'mean', device)
sufficiency_score = h.sufficiency_check(model, s_labels, s_tokens, s_answer_tokens, s_clean_logit_diff, 'mean', device)

100%|██████████| 11/11 [00:01<00:00,  6.85it/s]


In [40]:
h.plot_ablation(s_clean_logit_diff, necessity_score, sufficiency_score)

### Figure 5(b) robustness of the circuit

In [41]:
# numeric perturbed dataset
n_dataset = SyllogismDataset(
            N=N/6, # because of permutation
            seed=seed,
            device=device,
            type='numeric',
            template_type='AAA1'
        )
for s, l in zip(n_dataset.sentences[:6], n_dataset.labels[:6]):
    print(f'{s} => {l}')


All 8 are 1. All 1 are 7. Therefore, all 8 are =>  7
All 8 are 7. All 7 are 1. Therefore, all 8 are =>  1
All 1 are 8. All 8 are 7. Therefore, all 1 are =>  7
All 1 are 7. All 7 are 8. Therefore, all 1 are =>  8
All 7 are 8. All 8 are 1. Therefore, all 7 are =>  1
All 7 are 1. All 1 are 8. Therefore, all 7 are =>  8


In [42]:
# Corruption (option)
def perturb_quantifier(prompts, As, Bs, labels):
    candidates = [' Every', ' Every', ' Each', ' Each', ' All']
    corrupted_prompts = []
    for i in range(len(prompts)):
        prompt = prompts[i]
        label = labels[i]
        corrupted_labels = []
        A = As[i]
        B = Bs[i]
        new_list = list(filter(lambda x: x not in [A, B, label], candidates))
        target = random.sample(new_list, 2)
        a_be = ' are' if target[0] == ' All' else ' is'
        b_be = ' are' if target[1] == ' All' else ' is'
        corrupted_prompt = prompt.replace(' All' + B + ' are' , target[0] + B + a_be).replace('All' + A + ' are', target[1][1:] + A + b_be)
        corrupted_prompts.append(corrupted_prompt)
        corrupted_labels.append(target)

    return corrupted_prompts, corrupted_labels


In [43]:
n_prompts = n_dataset.sentences
n_labels = n_dataset.labels
n_second_labels = n_dataset.B

n_answers = list(zip(n_labels, n_second_labels))
n_answer_tokens = t.concat([
    model.to_tokens(names, prepend_bos=False).T for names in n_answers
])

In [45]:
q_prompts = s_dataset.sentences
q_labels = s_dataset.labels
q_As = s_dataset.A
q_Bs = s_dataset.B

q_corrupted_prompts, _ = perturb_quantifier(q_prompts, q_As, q_Bs, q_labels)

q_answers = list(zip(q_labels, q_Bs))
q_answer_tokens = t.concat([
    model.to_tokens(names, prepend_bos=False).T for names in q_answers
])

In [46]:
# tokenisation
n_tokens = model.to_tokens(n_prompts, prepend_bos=False).to(device)
q_tokens = model.to_tokens(q_prompts, prepend_bos=False).to(device)

In [47]:
# Compute logits for interventions
n_clean_logits, n_clean_cache = model.run_with_cache(n_tokens)
q_clean_logits, q_clean_cache = model.run_with_cache(n_tokens)

n_clean_logit_diff = h.compute_logit_diff(n_clean_logits, n_answer_tokens)
print(f"Clean logit diff: {n_clean_logit_diff:.4f}")

q_clean_logit_diff = h.compute_logit_diff(q_clean_logits, q_answer_tokens)
print(f"Clean logit diff: {q_clean_logit_diff:.4f}")

Clean logit diff: 0.1666
Clean logit diff: 0.0065


In [48]:
# necessity, sufficiency of numeric perturbation
n_necessity_score = h.necessity_check(model, n_labels, n_tokens, n_answer_tokens, n_clean_logit_diff, 'mean', device)
n_sufficiency_score = h.sufficiency_check(model, n_labels, n_tokens, n_answer_tokens, n_clean_logit_diff, 'mean', device)

100%|██████████| 11/11 [00:01<00:00,  6.90it/s]


In [49]:
# necessity, sufficiency of quantifier perturbation
q_necessity_score = h.necessity_check(model, q_labels, q_tokens, q_answer_tokens, q_clean_logit_diff, 'mean', device)
q_sufficiency_score = h.sufficiency_check(model, q_labels, q_tokens, q_answer_tokens, q_clean_logit_diff, 'mean', device)

100%|██████████| 11/11 [00:01<00:00,  6.89it/s]


In [50]:
h.plot_ablation_robust(y1=n_necessity_score, y2=n_sufficiency_score, y3=q_necessity_score, y4 = q_sufficiency_score,  baseline1= n_clean_logit_diff, baseline2=q_clean_logit_diff, title="")


# 5. Circuit Transferability

In [51]:
# setup
bc_prompts = bc_dataset.sentences
bc_labels = bc_dataset.labels
bc_second_labels = bc_dataset.B

bc_answers = list(zip(bc_labels, bc_second_labels))
bc_answer_tokens = t.concat([
    model.to_tokens(names, prepend_bos=False).T for names in bc_answers
])

bi_prompts = bi_dataset.sentences
bi_labels = bi_dataset.labels
bi_second_labels = bi_dataset.B

bi_answers = list(zip(bi_labels, bi_second_labels))
bi_answer_tokens = t.concat([
    model.to_tokens(names, prepend_bos=False).T for names in bi_answers
])

# tokenisation
bc_tokens = model.to_tokens(bc_prompts, prepend_bos=False).to(device)
bi_tokens = model.to_tokens(bi_prompts, prepend_bos=False).to(device)

In [52]:
# Compute logits for interventions
bc_clean_logits, bc_clean_cache = model.run_with_cache(bc_tokens)
bi_clean_logits, bi_clean_cache = model.run_with_cache(bi_tokens)

bc_clean_logit_diff = h.compute_logit_diff(bc_clean_logits, bc_answer_tokens)
print(f"Clean logit diff: {bc_clean_logit_diff:.4f}")

bi_clean_logit_diff = h.compute_logit_diff(bi_clean_logits, bi_answer_tokens)
print(f"Clean logit diff: {bi_clean_logit_diff:.4f}")

Clean logit diff: 0.2633
Clean logit diff: 0.5935


In [53]:
# belief-consistent
bc_necessity_score = h.necessity_check(model, bc_labels, bc_tokens, bc_answer_tokens, bc_clean_logit_diff, 'mean', device)
bc_sufficiency_score = h.sufficiency_check(model, bc_labels, bc_tokens, bc_answer_tokens, bc_clean_logit_diff, 'mean', device)

100%|██████████| 11/11 [00:01<00:00,  6.88it/s]


In [54]:
# belief-inconsistent
bi_necessity_score = h.necessity_check(model, bi_labels, bi_tokens, bi_answer_tokens, bi_clean_logit_diff, 'mean', device)
bi_sufficiency_score = h.sufficiency_check(model, bi_labels, bi_tokens, bi_answer_tokens, bi_clean_logit_diff, 'mean', device)

100%|██████████| 11/11 [00:01<00:00,  6.86it/s]


### Figure 6(a) belief-consistent

In [55]:
h.plot_ablation(bc_clean_logit_diff, bc_necessity_score, bc_sufficiency_score)

### Figure 6(b) belief-inconsistent

In [56]:
h.plot_ablation(bi_clean_logit_diff, bi_necessity_score, bi_sufficiency_score)

### Table 2 all unconditionally valid syllogisms

In [57]:
# ordered by accuracy
moods_ordered = ['AII3', 'IAI3', 'IAI4', 'AAA1', 'EAE1', 'EIO4', 'EIO3', 'AII1', 'AOO2', 'AEE4', 'OAO3', 'EIO1', 'EIO2', 'EAE2', 'AEE2']

In [59]:
# all symbolic datasets import
s_datasets = {}

for template in moods_ordered:
    s_dataset = SyllogismDataset(
            N=N,
            seed=seed,
            device=device,
            type='symbolic',
            template_type=template
        )
    s_datasets[template] = s_dataset


In [60]:
# setup
mood_dics = []

for template in moods_ordered:
    mood_dic = {
        'mood': template,
        'prompts': s_datasets[template].sentences,
        'labels': s_datasets[template].labels,
        'second_labels': s_datasets[template].B

    }
    s_answers = list(zip(mood_dic['labels'], mood_dic['second_labels']))
    mood_dic['answers'] = s_answers
    s_answer_tokens = t.concat([
        model.to_tokens(names, prepend_bos=False).T for names in s_answers
    ])
    mood_dic['answer_tokens'] = s_answer_tokens

    mood_dics.append(mood_dic)

In [61]:
# get base logit diff for 15 syllogisms
for i, template in enumerate(moods_ordered):
    tokens = model.to_tokens(mood_dics[i]['prompts'], prepend_bos=False).to(device)
    clean_logit_diff = h.get_batched_logit_diff(5, tokens, mood_dics[i]['answer_tokens'], model)
    mood_dics[i]['tokens'] = tokens
    mood_dics[i]['clean_logit_diff'] = clean_logit_diff

    bia_diff = 0
    for correct, wrong in mood_dics[i]['answer_tokens']:
        diff = model.unembed.b_U[correct.item()] - model.unembed.b_U[wrong.item()]
        bia_diff += diff

    bia_diff = bia_diff/len(mood_dics[i]['prompts'])
    mood_dics[i]['bia_diff'] = bia_diff
    print(mood_dics[i]['clean_logit_diff'], mood_dics[i]['bia_diff'])

tensor(1.1033, device='cuda:0') tensor(0., device='cuda:0')
tensor(0.4639, device='cuda:0') tensor(0., device='cuda:0')
tensor(0.4639, device='cuda:0') tensor(0., device='cuda:0')
tensor(0.4049, device='cuda:0') tensor(0., device='cuda:0')
tensor(0.1251, device='cuda:0') tensor(0., device='cuda:0')
tensor(0.4447, device='cuda:0') tensor(0., device='cuda:0')
tensor(0.4447, device='cuda:0') tensor(0., device='cuda:0')
tensor(-0.0879, device='cuda:0') tensor(0., device='cuda:0')
tensor(-0.8989, device='cuda:0') tensor(0., device='cuda:0')
tensor(-0.4749, device='cuda:0') tensor(0., device='cuda:0')
tensor(-0.4963, device='cuda:0') tensor(0., device='cuda:0')
tensor(-0.9533, device='cuda:0') tensor(0., device='cuda:0')
tensor(-1.2334, device='cuda:0') tensor(0., device='cuda:0')
tensor(-1.3570, device='cuda:0') tensor(0., device='cuda:0')
tensor(-1.4328, device='cuda:0') tensor(0., device='cuda:0')


In [62]:
# ablation
for i, template in enumerate(moods_ordered):
    mean_scores = h.necessity_check(model, mood_dics[i]['labels'], mood_dics[i]['tokens'], mood_dics[i]['answer_tokens'], mood_dics[i]['clean_logit_diff'], 'mean', device)
    sf_mean_scores = h.sufficiency_check(model, mood_dics[i]['labels'], mood_dics[i]['tokens'], mood_dics[i]['answer_tokens'], mood_dics[i]['clean_logit_diff'], 'mean',device)
    mood_dics[i]['mean_scores'] = mean_scores
    mood_dics[i]['sf_mean_score'] = sf_mean_scores

100%|██████████| 11/11 [00:10<00:00,  1.05it/s]


In [63]:
h.plot_ablation_syllogisms(mood_dics = mood_dics, title="")